# OverFitting UnderFitting

OverFitting -> On Trainig data. Classifier try to fit function / decision surface to have no error.

Sensitive to outliers


UnderFitting -> Classifier is lazy

# How to Determine the right K? 

Cross Validation -> Train data and using different k test on test data

Generalization : Algo performs well on the future unseen points

100% Data-> 60% Training ,Cross validation -> 20% , 20% -> Test Data

K Cross Validation

Problem with Cross Validation -> Losing 40 % data

Different apporcha-> Take 80% data and take k chunks of data

How to determine that we are not underfitting and overfitting

Accuary and Error


Train Error and Cross Validation Error are higher then underfit

Train Error is less and Cross Validation Error is high then overfit


# Time Based Splitting


There should be a time column and oldest 60% as train 

Time is a necesssary column


# KNN for regression
GET k nn values and get  mean or median

# Weighted KNN algorithm

weight = 1/distance of the point. Irrespective of max number based on the weight to distance whic are close

# vornoi diagram
Region Based

# Binary Search Tree

Used to search the element Time-> O(log(n))

# Build KD tree

projcet data to axis(2d assume)

In [1]:
# K-NN Cross validation 

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve,auc
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

In [2]:
con = sqlite3.connect('C:\\Users\\Mekakris\\Documents\\AC\\amazon-fine-food-reviews-data\\database.sqlite')

data = pd.read_sql_query("select * from Reviews where score!=3",con)

data=data[:3000]


def parition(x):
    if x<3:
        return "negative"
    return "positive"

actualscore=data['Score']


positiveNegative=actualscore.map(parition)
data['Score']=positiveNegative


In [3]:
# Data De duplication 

data=data.sort_values('ProductId',axis=0,ascending=True)
data=data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"},keep="first",inplace=False)
print(data.shape)

data=data[data.HelpfulnessNumerator<data.HelpfulnessDenominator]

print(data.shape)

data['Score'].value_counts()



(2991, 10)
(588, 10)


positive    343
negative    245
Name: Score, dtype: int64

In [4]:
# Clean Text 

import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop=set(stopwords.words('english'))
sno=nltk.stem.SnowballStemmer('english')

def cleanhtml(sentence): #function to clean the word of any html-tags\n",
        cleanr = re.compile('<.*?>')
        cleantext = re.sub(cleanr, ' ', sentence)
        return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters\n",
        cleaned = re.sub(r'[?|!|\'|\"|#]',r'',sentence)
        cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
        return  cleaned

#print(stop)



i=0
str1=' '
final_string=[]
all_positive_words=[] # store words from +ve reviews here\n",
all_negative_words=[] # store words from -ve reviews here.\n",
s=''

#data=data.iloc[0:1000]


for sent in data['Text'].values:
    filtered_sentence=[]
    #print(sent);\n",
    sent=cleanhtml(sent) # remove HTMl tags\n",
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (data['Score'].values)[i] == 'positive': 
                        all_positive_words.append(s) #list of all words used to describe positive reviews\n",
                    if(data['Score'].values)[i] == 'negative':
                        all_negative_words.append(s) #list of all words used to describe negative reviews reviews\n",
                else:
                    continue
            else:
               continue 
    #print(filtered_sentence)\n",
    str1 = b" ".join(filtered_sentence) #final string of cleaned words\n
    #print(\"***********************************************************************\")\n",
    final_string.append(str1)
    i+=1

    #rint(final_string)

In [5]:
data['CleanedText']=final_string


import sqlite3
import pandas as pd
import numpy as np
import nltk as nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve,auc
from nltk.stem.porter import PorterStemmer


#returns the object and convets text to vector
#count_vect = CountVectorizer() -> this is for normal text vector
#count_vect = CountVectorizer(ngram_range=(1,3))# -> this is for normal text vector
#final=count_vect.fit_transform(data['CleanedText'].values)

#print(final.shape)
type(data)

pandas.core.frame.DataFrame

In [6]:
from sklearn.preprocessing import StandardScaler

# Sort Data using time series

data = data.sort_values(['Time'])
lables=data['Score']

count_vect = TfidfVectorizer(ngram_range=(1,2))# -> this is for normal text vector
datavector=count_vect.fit_transform(data['CleanedText'].values)



In [7]:
from sklearn import cross_validation

# simple train test split
x_1, x_test, y_1, y_test = cross_validation.train_test_split( datavector, lables, test_size=0.33, random_state=42)
# x_1 has data. y_1 has train data lables

x_train,x_cv,y_train,y_cv = cross_validation.train_test_split(x_1,y_1,test_size=0.2,random_state=42)


#print(train_vector.shape)
#print(validation_vector.shape)
#print(y_cv.shape)
#print(type(train_vector))
#print(neigh.predict(x_test[5]))

print(x_train.shape,y_train.shape)
print(x_cv.shape,y_cv.shape)
#print(y_train.shape,y_test.shape)

(314, 23759) (314,)
(79, 23759) (79,)


C:\Users\Mekakris\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd


standardized_train_data = StandardScaler(with_mean=False).fit_transform(x_train)
standardized_test_data = StandardScaler(with_mean=False).fit_transform(x_cv)

#print(type(standardized_train_data))
#print(standardized_test_data.shape)
#print(y_train.shape)
#print(y_cv.shape)
for i in range(1,30,2):
    
 knn = KNeighborsClassifier(n_neighbors=i)

    # Train the the model
 knn.fit(standardized_train_data,y_train)
    
    # Predict
 pred = knn.predict(standardized_test_data)
    
    #Evaluate the accuracy
 acc=accuracy_score(y_cv,pred,normalize=True) * float(100)
    
 print('For k accuracy is %d',i,acc)




# TF-IDF


For k accuracy is %d 1 48.10126582278481
For k accuracy is %d 3 48.10126582278481
For k accuracy is %d 5 48.10126582278481
For k accuracy is %d 7 48.10126582278481
For k accuracy is %d 9 48.10126582278481
For k accuracy is %d 11 48.10126582278481
For k accuracy is %d 13 48.10126582278481
For k accuracy is %d 15 48.10126582278481
For k accuracy is %d 17 58.22784810126582
For k accuracy is %d 19 50.63291139240506
For k accuracy is %d 21 49.36708860759494
For k accuracy is %d 23 48.10126582278481
For k accuracy is %d 25 49.36708860759494
For k accuracy is %d 27 48.10126582278481
For k accuracy is %d 29 48.10126582278481


In [9]:
# NAIVE BAYES

from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.naive_bayes import GaussianNB


standardized_train_data = StandardScaler(with_mean=False).fit_transform(x_train)
standardized_test_data = StandardScaler(with_mean=False).fit_transform(x_cv)

print(standardized_train_data.shape,standardized_test_data.shape)

#print(type(standardized_train_data))
#print(standardized_test_data.shape)
#print(y_train.shape)
#print(y_cv.shape)
gnb = GaussianNB()
    # Train the the model

y_pred = gnb.fit(standardized_train_data.toarray(), y_train).predict(standardized_test_data.toarray())
    
print("Number of mislabeled points out of a total %d points : %d",(standardized_train_data.shape[0],(y_cv != y_pred).sum()))



# TF-IDF


(314, 23759) (79, 23759)
Number of mislabeled points out of a total %d points : %d (314, 28)


In [18]:
# K-Means

from sklearn.cluster import KMeans
import numpy as np


standardized_train_data = StandardScaler(with_mean=False).fit_transform(x_train)
standardized_test_data = StandardScaler(with_mean=False).fit_transform(x_cv)

print(standardized_train_data.shape,standardized_test_data.shape)

#print(type(standardized_train_data))
#print(standardized_test_data.shape)
#print(y_train.shape)
#print(y_cv.shape)
kmeans = KMeans(n_clusters=4, random_state=0).fit(standardized_train_data.toarray(),y_train)

y_pred =kmeans.predict(standardized_test_data.toarray())
    
print("Number of mislabeled points out of a total %d points : %d",(standardized_train_data.shape[0],(y_cv != y_pred).sum()))






(314, 23759) (79, 23759)
Number of mislabeled points out of a total %d points : %d (314, 79)


In [19]:
# DB-Scan-Means hold this. I need to work on this . this is incomplete

from sklearn.cluster import DBSCAN
import numpy as np


standardized_train_data = StandardScaler(with_mean=False).fit_transform(x_train)
standardized_test_data = StandardScaler(with_mean=False).fit_transform(x_cv)

print(standardized_train_data.shape,standardized_test_data.shape)

#print(type(standardized_train_data))
#print(standardized_test_data.shape)
#print(y_train.shape)
#print(y_cv.shape)
#dbscan =  DBSCAN(eps=0.3, min_samples=10).fit(standardized_train_data.toarray(),y_train)

#y_pred =dbscan.predict(standardized_test_data.toarray())
    
#print("Number of mislabeled points out of a total %d points : %d",(standardized_train_data.shape[0],(y_cv != y_pred).sum()))






(314, 23759) (79, 23759)


AttributeError: 'DBSCAN' object has no attribute 'predict'

In [20]:
# Decision Tree hold this. I need to work on this . this is incomplete

from sklearn import tree



standardized_train_data = StandardScaler(with_mean=False).fit_transform(x_train)
standardized_test_data = StandardScaler(with_mean=False).fit_transform(x_cv)

print(standardized_train_data.shape,standardized_test_data.shape)

#print(type(standardized_train_data))
#print(standardized_test_data.shape)
#print(y_train.shape)
#print(y_cv.shape)
tree=tree.DecisionTreeClassifier().fit(standardized_train_data.toarray(),y_train)

y_pred =tree.predict(standardized_test_data.toarray())
    
print("Number of mislabeled points out of a total %d points : %d",(standardized_train_data.shape[0],(y_cv != y_pred).sum()))






(314, 23759) (79, 23759)
Number of mislabeled points out of a total %d points : %d (314, 33)
